In [24]:
import sys
print("Python Version:", sys.version)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
import formulas as fm
pd.set_option('display.max_rows', 500)
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

Python Version: 3.8.3 (default, Jul  2 2020, 11:26:31) 
[Clang 10.0.0 ]


### Pulling the Data

In [25]:
# yymmdd format. Website: http://web.mta.info/developers/data/nyct/turnstile
dates = ['190803',
        '190810',
        '190817',
        '190824',
        '190831',
        '190907',
        '190914',
        '190921',
        '190928']

df_turns = fm.get_data(dates)

In [26]:
df_turns.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/27/2019,00:00:00,REGULAR,7148288,2418935
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/27/2019,04:00:00,REGULAR,7148306,2418939
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/27/2019,08:00:00,REGULAR,7148326,2418975
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/27/2019,12:00:00,REGULAR,7148441,2419042
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/27/2019,16:00:00,REGULAR,7148647,2419101


In [27]:
#Clean up the column Names
new_col_names = [name.strip() for name in df_turns.columns]
df_turns.columns = new_col_names


In [28]:
# Remove audited rows
df_turns=df_turns[df_turns.DESC!='RECOVR AUD']
# remove column
del df_turns['DESC']

### Clean up the Date Column

In [29]:
# Create a DateTime column that can be used for timeseries
df_turns = fm.Clean_Date(df_turns)

In [30]:
df_turns.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,EXITS,DATETIME,Weekday
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-07-27,1900-01-01 00:00:00,7148288,2418935,2019-07-27 00:00:00,Sat
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-07-27,1900-01-01 04:00:00,7148306,2418939,2019-07-27 04:00:00,Sat
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-07-27,1900-01-01 08:00:00,7148326,2418975,2019-07-27 08:00:00,Sat
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-07-27,1900-01-01 12:00:00,7148441,2419042,2019-07-27 12:00:00,Sat
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-07-27,1900-01-01 16:00:00,7148647,2419101,2019-07-27 16:00:00,Sat


In [31]:
#backup
df_backup=df_turns.copy()

In [32]:
#Sort in order of datetime
df_turns = df_turns.sort_values(by=['DATETIME']).reset_index()

In [33]:
# Caluclate Entries Aggregated at station level by DateTime
df_turns['Entry_Diff']=df_turns.groupby(['STATION', 'C/A', 'UNIT', 'SCP'],as_index=False)['ENTRIES'].transform(pd.Series.diff)['ENTRIES']

# Caluclate Exits Aggregated at station level by DateTime
df_turns['Exit_Diff']=df_turns.groupby(['STATION', 'C/A', 'UNIT', 'SCP'],as_index=False)['EXITS'].transform(pd.Series.diff)['EXITS']



In [34]:
#Absolute Value to deal with counting backward issues 
df_turns['Entry_Diff'] = abs(df_turns['Entry_Diff'])
df_turns['Exit_Diff']=abs(df_turns['Exit_Diff'])

# Calculate both
df_turns['Total_Traffic']=df_turns['Entry_Diff']+df_turns['Exit_Diff']

#### Adding weekday/weekend

In [35]:
df_turns['Day_Type'] = df_turns['Weekday'].apply(fm.daytype)
df_turns.head()

,index,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,EXITS,DATETIME,Weekday,Entry_Diff,Exit_Diff,Total_Traffic,Day_Type
0,0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-07-27,1900-01-01,7148288,2418935,2019-07-27,Sat,NaN,NaN,NaN,Weekend
1,112391,N605,R024,00-05-01,SUTPHIN-ARCHER,EJZ,IND,2019-07-27,1900-01-01,0,1168,2019-07-27,Sat,NaN,NaN,NaN,Weekend
2,112349,N605,R024,00-05-00,SUTPHIN-ARCHER,EJZ,IND,2019-07-27,1900-01-01,33556743,16777216,2019-07-27,Sat,NaN,NaN,NaN,Weekend
3,198692,R612,R057,01-03-03,ATL AV-BARCLAY,BDNQR2345,IRT,2019-07-27,1900-01-01,2120496,1957954,2019-07-27,Sat,NaN,NaN,NaN,Weekend
4,112307,N605,R024,00-00-06,SUTPHIN-ARCHER,EJZ,IND,2019-07-27,1900-01-01,5050187,2146519,2019-07-27,Sat,NaN,NaN,NaN,Weekend


#### Adding Time of Day column

In [36]:
df_turns['Day_Bucket'] = df_turns['TIME'].apply(fm.convertTimeBuckets)
df_turns.sample(5)

,index,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,EXITS,DATETIME,Weekday,Entry_Diff,Exit_Diff,Total_Traffic,Day_Type,Day_Bucket
465247,138205,R141,R031,00-00-00,34 ST-PENN STA,123,IRT,2019-08-11,1900-01-01 20:00:00,7193364,9442620,2019-08-11 20:00:00,Sun,322.0,435.0,757.0,Weekend,Evening
669536,167266,R258,R132,00-06-01,125 ST,456,IRT,2019-08-18,1900-01-01 17:00:00,7964118,7225349,2019-08-18 17:00:00,Sun,239.0,271.0,510.0,Weekend,Evening
936980,103269,N529,R257,00-00-00,EAST BROADWAY,F,IND,2019-08-27,1900-01-01 20:00:00,6369096,4985036,2019-08-27 20:00:00,Tue,570.0,607.0,1177.0,Weekday,Evening
485961,125501,PTH19,R549,02-01-03,NEWARK C,1,PTH,2019-08-12,1900-01-01 12:34:29,21659,763,2019-08-12 12:34:29,Mon,117.0,10.0,127.0,Weekday,Morning
1384087,99311,N508,R453,00-00-01,23 ST,FM,IND,2019-09-12,1900-01-01 04:00:00,675061413,522750272,2019-09-12 04:00:00,Thu,62.0,11.0,73.0,Weekday,Late Night


### Rami's Filtering 

In [37]:
# def Add_Weekday(data_frame, column='Date'):
#     dmap = {0:'Mon', 1: 'Tue', 2: 'Wed', 3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'}
#     #Convert column to date
#     data_frame[column] = pd.to_datetime(data_frame[column])
#     #Add column which shows the Weekday in integer
#     data_frame['Day_Number'] = data_frame[column].apply(lambda x: x.dayofweek)
#     #Add column which shows the Weekday in words
#     data_frame['Weekday'] = data_frame['Day_Number'].map(dmap)
#     return data_frame
# df_turns_backup2 = df_turns.copy()

In [38]:
#Filter out the Null Values
df_turns.dropna(subset=["Entry_Diff","Exit_Diff",'Total_Traffic'], axis=0, inplace=True)
df_turns.head()

,index,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,EXITS,DATETIME,Weekday,Entry_Diff,Exit_Diff,Total_Traffic,Day_Type,Day_Bucket
2554,118307,PTH03,R552,00-01-08,JOURNAL SQUARE,1,PTH,2019-07-27,1900-01-01 00:24:57,65,30,2019-07-27 00:24:57,Sat,2.0,0.0,2.0,Weekend,Late Night
2592,118308,PTH03,R552,00-01-08,JOURNAL SQUARE,1,PTH,2019-07-27,1900-01-01 00:38:57,65,30,2019-07-27 00:38:57,Sat,0.0,0.0,0.0,Weekend,Late Night
2636,118309,PTH03,R552,00-01-08,JOURNAL SQUARE,1,PTH,2019-07-27,1900-01-01 00:52:57,65,30,2019-07-27 00:52:57,Sat,0.0,0.0,0.0,Weekend,Late Night
2640,118310,PTH03,R552,00-01-08,JOURNAL SQUARE,1,PTH,2019-07-27,1900-01-01 00:56:25,4043,3513,2019-07-27 00:56:25,Sat,3978.0,3483.0,7461.0,Weekend,Late Night
4489,118311,PTH03,R552,00-01-08,JOURNAL SQUARE,1,PTH,2019-07-27,1900-01-01 01:06:57,65,30,2019-07-27 01:06:57,Sat,3978.0,3483.0,7461.0,Weekend,Late Night


In [39]:
dfc_grouped = df_turns.groupby(['DATE', 'STATION', 'Weekday'])['Entry_Diff', 'Exit_Diff', 'Total_Traffic'].sum().reset_index()

<ipython-input-39-41ea38c374a6>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dfc_grouped = df_turns.groupby(['DATE', 'STATION', 'Weekday'])['Entry_Diff', 'Exit_Diff', 'Total_Traffic'].sum().reset_index()


In [40]:
#The upper_quant_df outputs a dataframe of the upper 95% values for each station
upper_quant_df = dfc_grouped.groupby(['STATION'])['Total_Traffic'].quantile([.95]).reset_index()
#Then I merge it with the grouped data 
df_turns_no_outlier = dfc_grouped.merge(upper_quant_df, on='STATION')
df_turns_no_outlier.head()

,DATE,STATION,Weekday,Entry_Diff,Exit_Diff,Total_Traffic_x,level_1,Total_Traffic_y
0,2019-07-27,1 AV,Sat,4308.0,5146.0,9454.0,0.95,40228.0
1,2019-07-28,1 AV,Sun,4106.0,5309.0,9415.0,0.95,40228.0
2,2019-07-29,1 AV,Mon,14483.0,16525.0,31008.0,0.95,40228.0
3,2019-07-30,1 AV,Tue,16375.0,18206.0,34581.0,0.95,40228.0
4,2019-07-31,1 AV,Wed,16812.0,19777.0,36589.0,0.95,40228.0


In [41]:
#delete the rows where The difference between Total_Traffic_y and Total_Traffic_x < 0
df_turns_no_outlier['Diff'] = df_turns_no_outlier['Total_Traffic_y'] - df_turns_no_outlier['Total_Traffic_x']
df_turns_no_outlier = df_turns_no_outlier[df_turns_no_outlier['Diff']>= 0]

In [42]:
df_turns_no_outlier.sample(50)

,DATE,STATION,Weekday,Entry_Diff,Exit_Diff,Total_Traffic_x,level_1,Total_Traffic_y,Diff
19788,2019-08-16,PATH WTC 2,Fri,2712.0,11239.0,13951.0,0.95,17942.2,3991.2
3164,2019-08-10,25 AV,Sat,2999.0,789.0,3788.0,0.95,6820.7,3032.7
22100,2019-07-28,TOMPKINSVILLE,Sun,312.0,0.0,312.0,0.95,834.4,522.4
22075,2019-09-04,TIMES SQ-42 ST,Wed,101368.0,94389.0,195757.0,0.95,209555.5,13798.5
17222,2019-08-29,LONGWOOD AV,Thu,3134.0,3172.0,6306.0,0.95,7123.7,817.7
10857,2019-08-24,CANAL ST,Sat,49195.0,40024.0,89219.0,0.95,153639.1,64420.1
3700,2019-09-11,33 ST-RAWSON ST,Wed,14158.0,11500.0,25658.0,0.95,26312.1,654.1
2026,2019-08-06,18 ST,Tue,8496.0,5135.0,13631.0,0.95,15896.1,2265.1
5352,2019-07-27,59 ST,Sat,29981.0,26682.0,56663.0,0.95,127916.2,71253.2
23080,2019-09-01,WHITEHALL S-FRY,Sun,30.0,24.0,54.0,0.95,39472.1,39418.1


# End of group data cleaning 

The data should now have outliers removed and can be aggregated as necessary for analysis. The last cell with save a pickled dataframe for easy import into other notebooks.

In [43]:
df_turns_no_outlier.to_pickle('Cleaned_Working_Dataframe.pkl')